# High Train + High Validation Error: Data-Centric vs Model-Centric Experiments

## Objective
When both training and validation errors are high, systematically determine:
1. Should we focus on **data-centric** improvements first?
2. Or should we focus on **model-centric** improvements first?
3. What experiments definitively answer this question?

## The Dilemma

**Scenario**: Your model shows:
- Training Error: **HIGH** (e.g., 40% error rate)
- Validation Error: **HIGH** (e.g., 42% error rate)

This typically indicates **UNDERFITTING**, but the question is:
- Is it because the **data is insufficient/poor quality**?
- Or because the **model lacks capacity**?

## Key Hypothesis

**We argue: Start with DATA-CENTRIC experiments first**

**Why?**
1. **No model can learn from bad data** (garbage in, garbage out)
2. **Data issues are more common** than we think
3. **Data fixes are often cheaper** than architectural changes
4. **Good data + simple model** often beats bad data + complex model
5. **Model capacity only helps** if the signal exists in the data

## Approach
1. Create synthetic datasets with various quality issues
2. Show baseline performance (high train + val error)
3. Run data-centric experiments systematically
4. Run model-centric experiments for comparison
5. Demonstrate the decision framework

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif, SelectKBest, f_classif
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

# Set seeds
np.random.seed(42)
torch.manual_seed(42)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")

## Part 1: Synthetic Dataset with Multiple Quality Issues

We'll create a classification dataset that **intentionally** has data quality problems:

### Data Quality Issues to Simulate:
1. **Label Noise**: Incorrect labels (20% mislabeled)
2. **Feature Noise**: Irrelevant/random features (50% are noise)
3. **Missing Signal**: Weak correlation between features and target
4. **Class Imbalance**: Skewed class distribution
5. **Insufficient Features**: Important features not included

### True Underlying Pattern:
Despite these issues, there IS a learnable pattern (non-linear decision boundary).

This mimics real-world scenarios where:
- Data collection has errors
- Not all measured features are relevant
- The true signal is weak

In [ ]:
def generate_problematic_data(n_samples=1000, noise_level='high', random_state=42):
    """
    Generate classification data with various quality issues.
    
    True pattern: Non-linear decision boundary based on 3 key features
    Issues: Label noise, irrelevant features, weak signal
    """
    np.random.seed(random_state)
    
    # Generate informative features (only 3 truly matter)
    X_informative = np.random.randn(n_samples, 3)
    
    # True decision boundary: Non-linear combination
    # Class 1 if: X1^2 + X2 > 0 AND X3 > 0
    decision_value = X_informative[:, 0]**2 + X_informative[:, 1] + X_informative[:, 2]
    y_true = (decision_value > 0).astype(int)
    
    # Add label noise (corrupted labels)
    if noise_level == 'high':
        label_noise_rate = 0.25  # 25% wrong labels!
        feature_noise_ratio = 0.8  # 80% noise features
    elif noise_level == 'medium':
        label_noise_rate = 0.15
        feature_noise_ratio = 0.5
    else:  # low
        label_noise_rate = 0.05
        feature_noise_ratio = 0.2
    
    n_flip = int(label_noise_rate * n_samples)
    flip_indices = np.random.choice(n_samples, n_flip, replace=False)
    y_noisy = y_true.copy()
    y_noisy[flip_indices] = 1 - y_noisy[flip_indices]
    
    # Add many irrelevant features (noise features)
    n_noise_features = int(3 * feature_noise_ratio / (1 - feature_noise_ratio))
    X_noise = np.random.randn(n_samples, n_noise_features)
    
    # Combine informative and noise features
    X_full = np.hstack([X_informative, X_noise])
    
    # Add feature noise (random perturbations)
    X_full += np.random.normal(0, 0.5, X_full.shape)
    
    # Create class imbalance
    # Oversample class 1 to create 70-30 imbalance
    class_1_indices = np.where(y_noisy == 1)[0]
    n_oversample = int(len(class_1_indices) * 0.5)
    oversample_indices = np.random.choice(class_1_indices, n_oversample, replace=True)
    
    X_full = np.vstack([X_full, X_full[oversample_indices]])
    y_noisy = np.concatenate([y_noisy, y_noisy[oversample_indices]])
    y_true = np.concatenate([y_true, y_true[oversample_indices]])
    
    return X_full, y_noisy, y_true

# Generate problematic dataset
X, y_noisy, y_true = generate_problematic_data(n_samples=800, noise_level='high')

# Split data
X_train, X_temp, y_train_noisy, y_temp_noisy, y_train_true, y_temp_true = train_test_split(
    X, y_noisy, y_true, test_size=0.4, random_state=42, stratify=y_noisy
)

X_val, X_test, y_val_noisy, y_test_noisy, y_val_true, y_test_true = train_test_split(
    X_temp, y_temp_noisy, y_temp_true, test_size=0.5, random_state=42, stratify=y_temp_noisy
)

print(f"Dataset generated with quality issues:")
print(f"  Total features: {X.shape[1]} (only 3 are truly informative)")
print(f"  Training samples: {len(X_train)}")
print(f"  Validation samples: {len(X_val)}")
print(f"  Test samples: {len(X_test)}")
print(f"\nData quality issues:")
print(f"  Label noise rate: ~25%")
print(f"  Noise features: ~{X.shape[1] - 3} out of {X.shape[1]}")
print(f"  Class distribution (noisy labels): {np.bincount(y_noisy)}")
print(f"  Class distribution (true labels): {np.bincount(y_true)}")
print(f"  Label corruption in training: {np.mean(y_train_noisy != y_train_true)*100:.1f}%")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Feature correlation with target (using true labels)
ax = axes[0, 0]
correlations = []
for i in range(min(10, X_train.shape[1])):
    corr = np.corrcoef(X_train[:, i], y_train_true)[0, 1]
    correlations.append(abs(corr))

ax.bar(range(len(correlations)), correlations, color='steelblue', alpha=0.7)
ax.axhline(0.1, color='red', linestyle='--', linewidth=2, label='Weak correlation threshold')
ax.set_xlabel('Feature Index', fontsize=11)
ax.set_ylabel('|Correlation| with True Target', fontsize=11)
ax.set_title('Feature Relevance\n(Most features are irrelevant!)', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# 2. Label noise visualization
ax = axes[0, 1]
confusion_noisy = confusion_matrix(y_train_true, y_train_noisy)
sns.heatmap(confusion_noisy, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=True)
ax.set_xlabel('Noisy Labels', fontsize=11)
ax.set_ylabel('True Labels', fontsize=11)
ax.set_title(f'Label Noise\n({np.mean(y_train_noisy != y_train_true)*100:.1f}% corrupted)', 
             fontsize=12, fontweight='bold')

# 3. Class imbalance
ax = axes[0, 2]
class_counts = np.bincount(y_train_noisy)
ax.bar(['Class 0', 'Class 1'], class_counts, color=['coral', 'skyblue'], alpha=0.7, edgecolor='black', linewidth=2)
ax.set_ylabel('Sample Count', fontsize=11)
ax.set_title(f'Class Imbalance\n({class_counts[1]/sum(class_counts)*100:.1f}% are Class 1)', 
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# 4. PCA visualization with true labels
ax = axes[1, 0]
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
scatter = ax.scatter(X_train_pca[:, 0], X_train_pca[:, 1], 
                     c=y_train_true, cmap='coolwarm', s=30, alpha=0.6, edgecolors='black', linewidth=0.3)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontsize=11)
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontsize=11)
ax.set_title('PCA View (True Labels)\n(Low variance explained = weak signal)', fontsize=12, fontweight='bold')
plt.colorbar(scatter, ax=ax, label='True Class')

# 5. PCA with noisy labels
ax = axes[1, 1]
scatter = ax.scatter(X_train_pca[:, 0], X_train_pca[:, 1], 
                     c=y_train_noisy, cmap='coolwarm', s=30, alpha=0.6, edgecolors='black', linewidth=0.3)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontsize=11)
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontsize=11)
ax.set_title('PCA View (Noisy Labels)\n(Even more mixed up!)', fontsize=12, fontweight='bold')
plt.colorbar(scatter, ax=ax, label='Noisy Class')

# 6. Feature variance
ax = axes[1, 2]
feature_vars = np.var(X_train, axis=0)[:10]
ax.bar(range(len(feature_vars)), feature_vars, color='lightgreen', alpha=0.7, edgecolor='black')
ax.set_xlabel('Feature Index', fontsize=11)
ax.set_ylabel('Variance', fontsize=11)
ax.set_title('Feature Variability\n(First 10 features)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Data Quality Issues Visualization', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nKEY OBSERVATIONS:")
print("1. Most features have weak correlation with target")
print("2. 25% of labels are wrong")
print("3. Classes are imbalanced")
print("4. Low variance explained by PCA (weak signal)")
print("5. This is REALISTIC for many real-world datasets!")

## Part 2: Establish Baseline (The Problem)

Let's train a standard model and confirm we have high train + high validation error.

We'll use:
- **Simple model**: Logistic Regression
- **Medium model**: Random Forest
- **Complex model**: Gradient Boosting

**Expected Result**: All models should show high error on BOTH train and validation sets, indicating underfitting.

In [ ]:
def evaluate_model(model, X_train, y_train, X_val, y_val, model_name):
    """Train and evaluate a model"""
    model.fit(X_train, y_train)
    
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    
    train_acc = accuracy_score(y_train, train_pred)
    val_acc = accuracy_score(y_val, val_pred)
    
    train_err = 1 - train_acc
    val_err = 1 - val_acc
    
    return {
        'model_name': model_name,
        'model': model,
        'train_acc': train_acc,
        'val_acc': val_acc,
        'train_err': train_err,
        'val_err': val_err,
        'train_pred': train_pred,
        'val_pred': val_pred
    }

# Train baseline models
print("="*80)
print("BASELINE MODELS (Using Noisy Data As-Is)")
print("="*80)

baseline_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42)
}

baseline_results = {}

for name, model in baseline_models.items():
    print(f"\n{name}:")
    result = evaluate_model(model, X_train, y_train_noisy, X_val, y_val_noisy, name)
    baseline_results[name] = result
    
    print(f"  Training Error:   {result['train_err']:.2%}")
    print(f"  Validation Error: {result['val_err']:.2%}")
    print(f"  Gap: {abs(result['val_err'] - result['train_err']):.2%}")

# Visualize baseline performance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Error rates
ax = axes[0]
model_names = list(baseline_results.keys())
train_errs = [baseline_results[m]['train_err'] for m in model_names]
val_errs = [baseline_results[m]['val_err'] for m in model_names]

x_pos = np.arange(len(model_names))
width = 0.35

bars1 = ax.bar(x_pos - width/2, train_errs, width, label='Training Error', 
               color='blue', alpha=0.7, edgecolor='black', linewidth=2)
bars2 = ax.bar(x_pos + width/2, val_errs, width, label='Validation Error', 
               color='red', alpha=0.7, edgecolor='black', linewidth=2)

ax.axhline(0.25, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='High Error Threshold')
ax.set_ylabel('Error Rate', fontsize=13)
ax.set_xlabel('Model Type', fontsize=13)
ax.set_title('Baseline Performance: HIGH Errors on Both Sets!', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, rotation=15, ha='right')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1%}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Confusion matrix for best model
ax = axes[1]
best_model_name = min(baseline_results.keys(), key=lambda k: baseline_results[k]['val_err'])
best_result = baseline_results[best_model_name]
cm = confusion_matrix(y_val_noisy, best_result['val_pred'])

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=True)
ax.set_xlabel('Predicted Label', fontsize=12)
ax.set_ylabel('True Label (Noisy)', fontsize=12)
ax.set_title(f'Confusion Matrix: {best_model_name}\n(Best baseline model)', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("PROBLEM CONFIRMED: High training AND validation errors!")
print("This indicates UNDERFITTING.")
print("\nNow the question: Fix data first or fix model first?")
print("="*80)

## Part 3: DATA-CENTRIC Experiments (Run These FIRST!)

### Why Data-Centric First?

1. **Fundamental Principle**: Models can only learn patterns that exist in the data
2. **Cost-Effective**: Data cleaning is usually cheaper than model development
3. **Broader Impact**: Better data helps ALL models
4. **Common Issue**: Data quality problems are more common than insufficient model capacity

### Data-Centric Experiments to Run:

1. **Label Noise Detection & Cleaning**
   - Identify potentially mislabeled examples
   - Clean or remove suspicious labels

2. **Feature Engineering**
   - Create interaction features
   - Polynomial features
   - Domain-specific transformations

3. **Feature Selection**
   - Remove irrelevant/noisy features
   - Keep only informative features

4. **Data Augmentation**
   - Generate more training samples
   - Synthetic examples

5. **Class Balancing**
   - Address class imbalance
   - Resampling strategies

6. **Data Quality Analysis**
   - Check for duplicates
   - Handle outliers
   - Verify data integrity

In [ ]:
print("="*80)
print("DATA-CENTRIC EXPERIMENT 1: Label Noise Detection & Cleaning")
print("="*80)

def detect_label_noise(X, y, model, threshold=0.3):
    """
    Detect potentially mislabeled examples using model confidence.
    Strategy: Train model, find examples where it's very confident but wrong.
    """
    # Train model
    model.fit(X, y)
    
    # Get prediction probabilities
    if hasattr(model, 'predict_proba'):
        proba = model.predict_proba(X)
        # Get confidence in predicted class
        pred_class = np.argmax(proba, axis=1)
        confidence = np.max(proba, axis=1)
        
        # Examples where model is confident but wrong
        is_wrong = (pred_class != y)
        is_confident = (confidence > (1 - threshold))
        
        suspicious_indices = np.where(is_wrong & is_confident)[0]
        
        return suspicious_indices, confidence
    else:
        return np.array([]), None

# Use cross-validation-like approach to detect noise
from sklearn.model_selection import KFold

print("\nDetecting mislabeled examples...")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
suspicious_counts = np.zeros(len(X_train))

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    X_fold_train = X_train[train_idx]
    y_fold_train = y_train_noisy[train_idx]
    X_fold_val = X_train[val_idx]
    y_fold_val = y_train_noisy[val_idx]
    
    # Train model on fold
    model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
    model.fit(X_fold_train, y_fold_train)
    
    # Check validation fold
    if hasattr(model, 'predict_proba'):
        proba = model.predict_proba(X_fold_val)
        pred_class = np.argmax(proba, axis=1)
        confidence = np.max(proba, axis=1)
        
        is_wrong = (pred_class != y_fold_val)
        is_confident = (confidence > 0.7)
        
        suspicious_in_fold = is_wrong & is_confident
        suspicious_counts[val_idx] += suspicious_in_fold

# Identify consistently suspicious examples
noise_threshold = 3  # Flagged in at least 3/5 folds
suspected_noise_indices = np.where(suspicious_counts >= noise_threshold)[0]

print(f"Suspected mislabeled examples: {len(suspected_noise_indices)} / {len(X_train)}")
print(f"Actual mislabeled examples: {np.sum(y_train_noisy != y_train_true)}")
print(f"Overlap: {np.sum(np.isin(suspected_noise_indices, np.where(y_train_noisy != y_train_true)[0]))} correctly identified")

# Create cleaned dataset
X_train_cleaned = np.delete(X_train, suspected_noise_indices, axis=0)
y_train_cleaned = np.delete(y_train_noisy, suspected_noise_indices)

print(f"\nCleaned training set size: {len(X_train_cleaned)} (removed {len(suspected_noise_indices)} examples)")

# Evaluate with cleaned labels
print("\nResults after label cleaning:")
cleaned_results = {}

for name, model_class in baseline_models.items():
    model = model_class
    result = evaluate_model(model, X_train_cleaned, y_train_cleaned, X_val, y_val_noisy, name)
    cleaned_results[name] = result
    
    print(f"\n{name}:")
    print(f"  Training Error:   {result['train_err']:.2%} (was {baseline_results[name]['train_err']:.2%})")
    print(f"  Validation Error: {result['val_err']:.2%} (was {baseline_results[name]['val_err']:.2%})")
    
    improvement = baseline_results[name]['val_err'] - result['val_err']
    print(f"  Improvement: {improvement:.2%}")

print("\n" + "="*80)

In [ ]:
print("="*80)
print("DATA-CENTRIC EXPERIMENT 2: Feature Engineering")
print("="*80)

print("\nCreating polynomial and interaction features...")

# Create polynomial features (degree 2)
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)

X_train_poly = poly.fit_transform(X_train)
X_val_poly = poly.transform(X_val)

print(f"Original features: {X_train.shape[1]}")
print(f"After polynomial expansion: {X_train_poly.shape[1]}")

# Note: Too many features can be bad, so let's also select the best ones
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Select top features based on mutual information
k_best = min(20, X_train_poly.shape[1])  # Select top 20 features
selector = SelectKBest(mutual_info_classif, k=k_best)

X_train_poly_selected = selector.fit_transform(X_train_poly, y_train_noisy)
X_val_poly_selected = selector.transform(X_val_poly)

print(f"After feature selection: {X_train_poly_selected.shape[1]}")

# Evaluate with engineered features
print("\nResults after feature engineering:")
poly_results = {}

for name, model_class in baseline_models.items():
    model = model_class
    result = evaluate_model(model, X_train_poly_selected, y_train_noisy, 
                           X_val_poly_selected, y_val_noisy, name)
    poly_results[name] = result
    
    print(f"\n{name}:")
    print(f"  Training Error:   {result['train_err']:.2%} (was {baseline_results[name]['train_err']:.2%})")
    print(f"  Validation Error: {result['val_err']:.2%} (was {baseline_results[name]['val_err']:.2%})")
    
    improvement = baseline_results[name]['val_err'] - result['val_err']
    print(f"  Improvement: {improvement:.2%}")

print("\n" + "="*80)

In [ ]:
print("="*80)
print("DATA-CENTRIC EXPERIMENT 3: Feature Selection (Remove Noise)")
print("="*80)

print("\nAnalyzing feature importance...")

# Use Random Forest to get feature importance
rf_temp = RandomForestClassifier(n_estimators=100, random_state=42)
rf_temp.fit(X_train, y_train_noisy)
feature_importance = rf_temp.feature_importances_

# Also use mutual information
mi_scores = mutual_info_classif(X_train, y_train_noisy, random_state=42)

# Combine both metrics
combined_score = (feature_importance + mi_scores) / 2

# Select top features
n_select = 5  # We know only 3 are truly informative, but be a bit generous
top_feature_indices = np.argsort(combined_score)[-n_select:]

print(f"Selected top {n_select} features from {X_train.shape[1]} total features")
print(f"Top feature indices: {top_feature_indices}")
print(f"Are the first 3 features included? {all(i in top_feature_indices for i in [0, 1, 2])}")

# Create dataset with selected features only
X_train_selected = X_train[:, top_feature_indices]
X_val_selected = X_val[:, top_feature_indices]

# Evaluate with selected features
print("\nResults after feature selection:")
selected_results = {}

for name, model_class in baseline_models.items():
    model = model_class
    result = evaluate_model(model, X_train_selected, y_train_noisy, 
                           X_val_selected, y_val_noisy, name)
    selected_results[name] = result
    
    print(f"\n{name}:")
    print(f"  Training Error:   {result['train_err']:.2%} (was {baseline_results[name]['train_err']:.2%})")
    print(f"  Validation Error: {result['val_err']:.2%} (was {baseline_results[name]['val_err']:.2%})")
    
    improvement = baseline_results[name]['val_err'] - result['val_err']
    print(f"  Improvement: {improvement:.2%}")

# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

ax = axes[0]
n_show = min(15, len(feature_importance))
indices = np.argsort(feature_importance)[-n_show:]
ax.barh(range(n_show), feature_importance[indices], color='steelblue', alpha=0.7)
ax.set_yticks(range(n_show))
ax.set_yticklabels([f'Feature {i}' for i in indices])
ax.set_xlabel('Importance Score', fontsize=12)
ax.set_title('Feature Importance (Random Forest)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Highlight truly informative features
for i, idx in enumerate(indices):
    if idx in [0, 1, 2]:
        ax.get_yticklabels()[i].set_color('red')
        ax.get_yticklabels()[i].set_fontweight('bold')

ax = axes[1]
indices_mi = np.argsort(mi_scores)[-n_show:]
ax.barh(range(n_show), mi_scores[indices_mi], color='coral', alpha=0.7)
ax.set_yticks(range(n_show))
ax.set_yticklabels([f'Feature {i}' for i in indices_mi])
ax.set_xlabel('Mutual Information Score', fontsize=12)
ax.set_title('Feature Relevance (Mutual Information)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

for i, idx in enumerate(indices_mi):
    if idx in [0, 1, 2]:
        ax.get_yticklabels()[i].set_color('red')
        ax.get_yticklabels()[i].set_fontweight('bold')

plt.suptitle('Feature Analysis (Red = Truly Informative Features)', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n" + "="*80)

In [ ]:
print("="*80)
print("DATA-CENTRIC EXPERIMENT 4: Combining All Improvements")
print("="*80)

print("\nApplying multiple data-centric improvements together:")
print("  1. Label noise cleaning")
print("  2. Feature selection")
print("  3. Feature engineering (on selected features)")

# Step 1: Clean labels (remove suspected noise)
X_train_v1 = np.delete(X_train, suspected_noise_indices, axis=0)
y_train_v1 = np.delete(y_train_noisy, suspected_noise_indices)

# Step 2: Feature selection (keep only informative features)
X_train_v2 = X_train_v1[:, top_feature_indices]
X_val_v2 = X_val[:, top_feature_indices]

# Step 3: Engineer features on the selected features
poly_selected = PolynomialFeatures(degree=2, include_bias=False)
X_train_v3 = poly_selected.fit_transform(X_train_v2)
X_val_v3 = poly_selected.transform(X_val_v2)

print(f"\nFinal training set:")
print(f"  Samples: {len(X_train_v3)} (removed {len(suspected_noise_indices)} noisy)")
print(f"  Features: {X_train_v3.shape[1]} (selected + engineered from {X_train.shape[1]} original)")

# Evaluate combined approach
print("\nResults after combined data-centric improvements:")
combined_dc_results = {}

for name, model_class in baseline_models.items():
    model = model_class
    result = evaluate_model(model, X_train_v3, y_train_v1, X_val_v3, y_val_noisy, name)
    combined_dc_results[name] = result
    
    print(f"\n{name}:")
    print(f"  Training Error:   {result['train_err']:.2%} (was {baseline_results[name]['train_err']:.2%})")
    print(f"  Validation Error: {result['val_err']:.2%} (was {baseline_results[name]['val_err']:.2%})")
    
    improvement = baseline_results[name]['val_err'] - result['val_err']
    print(f"  Improvement: {improvement:.2%} ⭐")

print("\n" + "="*80)

## Part 4: MODEL-CENTRIC Experiments (For Comparison)

Now let's try model-centric approaches on the ORIGINAL data:

### Model-Centric Experiments:

1. **Increase Model Capacity**
   - Deeper networks
   - More parameters
   - More complex architectures

2. **Change Architecture**
   - Different model families
   - Ensemble methods

3. **Hyperparameter Tuning**
   - Learning rate
   - Regularization
   - Architecture parameters

**Hypothesis**: These will help less than data-centric improvements because the data quality is the bottleneck.

In [ ]:
print("="*80)
print("MODEL-CENTRIC EXPERIMENT 1: Increase Model Capacity")
print("="*80)

# Define more complex models
complex_models = {
    'Deep Tree (depth=20)': DecisionTreeClassifier(max_depth=20, random_state=42),
    'Large Random Forest (n=500)': RandomForestClassifier(n_estimators=500, max_depth=20, random_state=42),
    'Large Gradient Boosting (n=300)': GradientBoostingClassifier(n_estimators=300, max_depth=7, random_state=42)
}

print("\nTraining high-capacity models on ORIGINAL data...")
capacity_results = {}

for name, model in complex_models.items():
    print(f"\n{name}:")
    result = evaluate_model(model, X_train, y_train_noisy, X_val, y_val_noisy, name)
    capacity_results[name] = result
    
    # Compare to baseline
    baseline_name = name.split('(')[0].strip()
    if 'Deep Tree' in name:
        baseline_err = baseline_results['Logistic Regression']['val_err']
    elif 'Random Forest' in name:
        baseline_err = baseline_results['Random Forest']['val_err']
    else:
        baseline_err = baseline_results['Gradient Boosting']['val_err']
    
    print(f"  Training Error:   {result['train_err']:.2%}")
    print(f"  Validation Error: {result['val_err']:.2%}")
    print(f"  Improvement over baseline: {baseline_err - result['val_err']:.2%}")

print("\n" + "="*80)

In [ ]:
print("="*80)
print("MODEL-CENTRIC EXPERIMENT 2: Deep Neural Network")
print("="*80)

class DeepNN(nn.Module):
    """Deep neural network with many layers"""
    def __init__(self, input_dim, hidden_dims=[128, 64, 32, 16], output_dim=2, dropout=0.3):
        super(DeepNN, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def train_neural_network(model, X_train, y_train, X_val, y_val, epochs=100, lr=0.001, batch_size=32):
    """Train neural network"""
    # Prepare data
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.LongTensor(y_train)
    X_val_tensor = torch.FloatTensor(X_val)
    y_val_tensor = torch.LongTensor(y_val)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
        
        if (epoch + 1) % 20 == 0:
            model.eval()
            with torch.no_grad():
                train_outputs = model(X_train_tensor)
                val_outputs = model(X_val_tensor)
                
                _, train_pred = torch.max(train_outputs, 1)
                _, val_pred = torch.max(val_outputs, 1)
                
                train_acc = (train_pred == y_train_tensor).float().mean().item()
                val_acc = (val_pred == y_val_tensor).float().mean().item()
                
                print(f"  Epoch {epoch+1}: Train Acc={train_acc:.3f}, Val Acc={val_acc:.3f}")
    
    # Final evaluation
    model.eval()
    with torch.no_grad():
        train_outputs = model(X_train_tensor)
        val_outputs = model(X_val_tensor)
        
        _, train_pred = torch.max(train_outputs, 1)
        _, val_pred = torch.max(val_outputs, 1)
        
        train_acc = (train_pred == y_train_tensor).float().mean().item()
        val_acc = (val_pred == y_val_tensor).float().mean().item()
    
    return train_acc, val_acc

# Train deep neural network
print("\nTraining deep neural network on ORIGINAL data...")
print(f"Input dimension: {X_train.shape[1]}")

deep_nn = DeepNN(input_dim=X_train.shape[1], hidden_dims=[128, 64, 32, 16])
train_acc, val_acc = train_neural_network(deep_nn, X_train, y_train_noisy, X_val, y_val_noisy, 
                                          epochs=100, lr=0.001)

print(f"\nFinal Results:")
print(f"  Training Error:   {1-train_acc:.2%}")
print(f"  Validation Error: {1-val_acc:.2%}")
print(f"  Improvement over baseline: {baseline_results['Logistic Regression']['val_err'] - (1-val_acc):.2%}")

print("\n" + "="*80)

In [ ]:
print("="*80)
print("COMPREHENSIVE COMPARISON: Data-Centric vs Model-Centric")
print("="*80)

# Collect all results
comparison_data = {
    'Baseline (Raw Data + Simple Model)': {
        'approach': 'Baseline',
        'val_err': baseline_results['Logistic Regression']['val_err']
    },
    'Label Cleaning': {
        'approach': 'Data-Centric',
        'val_err': cleaned_results['Logistic Regression']['val_err']
    },
    'Feature Engineering': {
        'approach': 'Data-Centric',
        'val_err': poly_results['Logistic Regression']['val_err']
    },
    'Feature Selection': {
        'approach': 'Data-Centric',
        'val_err': selected_results['Logistic Regression']['val_err']
    },
    'Combined Data-Centric': {
        'approach': 'Data-Centric',
        'val_err': combined_dc_results['Logistic Regression']['val_err']
    },
    'High Capacity Model': {
        'approach': 'Model-Centric',
        'val_err': capacity_results['Large Random Forest (n=500)']['val_err']
    },
    'Deep Neural Network': {
        'approach': 'Model-Centric',
        'val_err': 1 - val_acc
    }
}

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Plot 1: Bar chart comparison
ax = axes[0, 0]
names = list(comparison_data.keys())
errors = [comparison_data[n]['val_err'] for n in names]
colors_bars = ['gray' if comparison_data[n]['approach'] == 'Baseline' else 
               'green' if comparison_data[n]['approach'] == 'Data-Centric' else 
               'blue' for n in names]

bars = ax.barh(range(len(names)), errors, color=colors_bars, alpha=0.7, edgecolor='black', linewidth=2)
ax.set_yticks(range(len(names)))
ax.set_yticklabels(names, fontsize=10)
ax.set_xlabel('Validation Error Rate', fontsize=12)
ax.set_title('Validation Error: All Approaches', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
ax.invert_yaxis()

# Add value labels
for i, (bar, err) in enumerate(zip(bars, errors)):
    ax.text(err + 0.01, i, f'{err:.1%}', va='center', fontsize=10, fontweight='bold')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='gray', alpha=0.7, label='Baseline'),
    Patch(facecolor='green', alpha=0.7, label='Data-Centric'),
    Patch(facecolor='blue', alpha=0.7, label='Model-Centric')
]
ax.legend(handles=legend_elements, fontsize=11, loc='lower right')

# Plot 2: Improvement over baseline
ax = axes[0, 1]
baseline_err = comparison_data['Baseline (Raw Data + Simple Model)']['val_err']
improvements = [(baseline_err - comparison_data[n]['val_err']) for n in names[1:]]
names_imp = names[1:]

colors_imp = ['green' if comparison_data[n]['approach'] == 'Data-Centric' else 'blue' 
              for n in names_imp]

bars = ax.barh(range(len(names_imp)), improvements, color=colors_imp, alpha=0.7, 
               edgecolor='black', linewidth=2)
ax.set_yticks(range(len(names_imp)))
ax.set_yticklabels(names_imp, fontsize=10)
ax.set_xlabel('Improvement over Baseline (Error Reduction)', fontsize=12)
ax.set_title('Absolute Improvement in Error Rate', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
ax.axvline(0, color='red', linestyle='--', linewidth=2)
ax.invert_yaxis()

# Add value labels
for i, (bar, imp) in enumerate(zip(bars, improvements)):
    ax.text(imp + 0.005, i, f'{imp:.2%}', va='center', fontsize=10, fontweight='bold')

# Plot 3: Categorized comparison
ax = axes[1, 0]

dc_experiments = ['Label Cleaning', 'Feature Engineering', 'Feature Selection', 'Combined Data-Centric']
mc_experiments = ['High Capacity Model', 'Deep Neural Network']

dc_errors = [comparison_data[n]['val_err'] for n in dc_experiments]
mc_errors = [comparison_data[n]['val_err'] for n in mc_experiments]

x_dc = np.arange(len(dc_experiments))
x_mc = np.arange(len(mc_experiments))

bars1 = ax.bar(x_dc - 0.2, dc_errors, 0.4, label='Data-Centric', 
               color='green', alpha=0.7, edgecolor='black', linewidth=2)
ax.axhline(baseline_err, color='gray', linestyle='--', linewidth=2, alpha=0.7, label='Baseline')

ax.set_ylabel('Validation Error Rate', fontsize=12)
ax.set_title('Data-Centric Experiments', fontsize=13, fontweight='bold')
ax.set_xticks(x_dc)
ax.set_xticklabels([n.replace(' ', '\n') for n in dc_experiments], fontsize=9)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1%}', ha='center', va='bottom', fontsize=9, fontweight='bold')

ax = axes[1, 1]
bars2 = ax.bar(x_mc, mc_errors, 0.5, label='Model-Centric', 
               color='blue', alpha=0.7, edgecolor='black', linewidth=2)
ax.axhline(baseline_err, color='gray', linestyle='--', linewidth=2, alpha=0.7, label='Baseline')

ax.set_ylabel('Validation Error Rate', fontsize=12)
ax.set_title('Model-Centric Experiments', fontsize=13, fontweight='bold')
ax.set_xticks(x_mc)
ax.set_xticklabels([n.replace(' ', '\n') for n in mc_experiments], fontsize=9)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

for bar in bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1%}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.suptitle('COMPREHENSIVE COMPARISON: Data-Centric vs Model-Centric Approaches', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*80)
print("SUMMARY OF RESULTS")
print("="*80)

print(f"\nBaseline Error: {baseline_err:.2%}")
print("\nData-Centric Approaches:")
for name in dc_experiments:
    err = comparison_data[name]['val_err']
    improvement = baseline_err - err
    print(f"  {name:30s}: {err:.2%} (↓ {improvement:.2%})")

print("\nModel-Centric Approaches:")
for name in mc_experiments:
    err = comparison_data[name]['val_err']
    improvement = baseline_err - err
    print(f"  {name:30s}: {err:.2%} (↓ {improvement:.2%})")

# Calculate average improvements
avg_dc_improvement = np.mean([(baseline_err - comparison_data[n]['val_err']) 
                               for n in dc_experiments])
avg_mc_improvement = np.mean([(baseline_err - comparison_data[n]['val_err']) 
                               for n in mc_experiments])

print(f"\nAverage Improvement:")
print(f"  Data-Centric: {avg_dc_improvement:.2%}")
print(f"  Model-Centric: {avg_mc_improvement:.2%}")

if avg_dc_improvement > avg_mc_improvement:
    print(f"\n✓ Data-Centric approaches performed {(avg_dc_improvement/avg_mc_improvement - 1)*100:.1f}% better on average!")
else:
    print(f"\n✓ Model-Centric approaches performed {(avg_mc_improvement/avg_dc_improvement - 1)*100:.1f}% better on average!")

print("="*80)

## Part 5: Decision Framework

### When to Prioritize Data-Centric vs Model-Centric

Based on our experiments, here's a systematic decision framework:

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Decision Tree
ax = axes[0, 0]
ax.axis('off')
ax.set_title('Decision Framework', fontsize=14, fontweight='bold', pad=20)

decision_text = """
WHEN BOTH TRAIN & VAL ERRORS ARE HIGH:

Step 1: Quick Data Quality Check
├─ Label noise? (contradictory examples)
├─ Feature relevance? (correlation with target)
├─ Class imbalance?
└─ Missing features? (domain knowledge)

IF data issues found → DATA-CENTRIC FIRST
├─ Clean labels
├─ Engineer features
├─ Select relevant features
├─ Balance classes
└─ Then reassess

IF no obvious data issues → Check model
├─ Is model too simple? (linear for non-linear)
├─ Training converged?
└─ Gradients healthy?

Step 2: After Initial Fix
├─ Errors still high?
│  ├─ Try the OTHER approach
│  └─ Or combine both
└─ Errors acceptable?
   └─ Done! Monitor & deploy

RULE OF THUMB:
Data-centric FIRST because:
✓ Can't learn from bad data
✓ Benefits ALL models
✓ Usually faster/cheaper
✓ More common issue
"""

ax.text(0.05, 0.95, decision_text, transform=ax.transAxes,
        fontsize=9, verticalalignment='top', family='monospace',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

# Data Quality Checklist
ax = axes[0, 1]
ax.axis('off')
ax.set_title('Data Quality Checklist', fontsize=14, fontweight='bold', pad=20)

checklist_text = """
BEFORE INCREASING MODEL CAPACITY:

☐ Label Quality
  • Any contradictory examples?
  • Labeling guidelines clear?
  • Multiple annotators agree?
  • Spot-check random samples

☐ Feature Relevance
  • Correlation with target?
  • Domain expertise consulted?
  • Missing important features?
  • Too many irrelevant features?

☐ Data Distribution
  • Class balance appropriate?
  • Train/val/test similar?
  • Outliers handled?
  • Missing values addressed?

☐ Data Sufficiency
  • Enough samples per class?
  • Covers all scenarios?
  • Representative of production?

☐ Feature Engineering
  • Interactions captured?
  • Non-linear transforms?
  • Domain-specific features?

✓ If ANY issues → Fix data first!
✗ If ALL clean → Consider model
"""

ax.text(0.05, 0.95, checklist_text, transform=ax.transAxes,
        fontsize=9, verticalalignment='top', family='monospace',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

# Cost-Benefit Analysis
ax = axes[1, 0]
ax.axis('off')
ax.set_title('Cost-Benefit Analysis', fontsize=14, fontweight='bold', pad=20)

cost_text = """
DATA-CENTRIC APPROACHES:

Pros:
  ✓ Benefits all models
  ✓ Usually faster to implement
  ✓ Often cheaper (vs GPU time)
  ✓ Provides insights into problem
  ✓ Improvements tend to compound

Cons:
  ✗ May need domain expertise
  ✗ Manual cleaning can be tedious
  ✗ Hard to fully automate

Typical Cost: 1-5 days
Typical Gain: 5-20% error reduction


MODEL-CENTRIC APPROACHES:

Pros:
  ✓ Well-documented techniques
  ✓ Can be automated
  ✓ Lots of tools available

Cons:
  ✗ Expensive (compute)
  ✗ Slower experimentation
  ✗ Diminishing returns
  ✗ Won't fix bad data

Typical Cost: 3-10 days + compute
Typical Gain: 2-10% error reduction
               (IF data is good)
"""

ax.text(0.05, 0.95, cost_text, transform=ax.transAxes,
        fontsize=9, verticalalignment='top', family='monospace',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))

# Examples
ax = axes[1, 1]
ax.axis('off')
ax.set_title('Real-World Examples', fontsize=14, fontweight='bold', pad=20)

examples_text = """
WHEN DATA-CENTRIC WON:

Example 1: Medical Diagnosis
  Problem: 40% error on X-ray classification
  Data fix: Removed mislabeled scans (10%)
  Result: Error → 25% (simple model!)
  
Example 2: Fraud Detection
  Problem: 35% false positives
  Data fix: Added transaction time features
  Result: Error → 18% (same model)

Example 3: NLP Sentiment
  Problem: 45% error on reviews
  Data fix: Cleaned HTML, deduped
  Result: Error → 30% (no model change)


WHEN MODEL-CENTRIC WON:

Example 4: Image Recognition
  Problem: 30% error on faces
  Data: Clean, sufficient, diverse
  Model fix: CNN instead of SVM
  Result: Error → 10%

Example 5: Time Series
  Problem: 25% error on forecasting
  Data: Clean, complete
  Model fix: LSTM instead of ARIMA
  Result: Error → 12%


LESSON: Check data FIRST, but both
        approaches often needed!
"""

ax.text(0.05, 0.95, examples_text, transform=ax.transAxes,
        fontsize=8.5, verticalalignment='top', family='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('PRACTICAL DECISION FRAMEWORK', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Create a systematic experiment priority ranking
print("="*80)
print("EXPERIMENT PRIORITY RANKING")
print("="*80)

experiments = [
    {
        'name': 'Label Noise Detection',
        'type': 'Data-Centric',
        'priority': 1,
        'cost': 'Low',
        'expected_gain': 'High',
        'time': '1-2 hours',
        'difficulty': 'Easy',
        'description': 'Identify and remove/fix mislabeled examples'
    },
    {
        'name': 'Feature Relevance Analysis',
        'type': 'Data-Centric',
        'priority': 2,
        'cost': 'Low',
        'expected_gain': 'High',
        'time': '1-3 hours',
        'difficulty': 'Easy',
        'description': 'Identify and remove irrelevant features'
    },
    {
        'name': 'Basic Feature Engineering',
        'type': 'Data-Centric',
        'priority': 3,
        'cost': 'Low',
        'expected_gain': 'Medium-High',
        'time': '2-4 hours',
        'difficulty': 'Medium',
        'description': 'Create polynomial/interaction features'
    },
    {
        'name': 'Class Balancing',
        'type': 'Data-Centric',
        'priority': 4,
        'cost': 'Low',
        'expected_gain': 'Medium',
        'time': '1 hour',
        'difficulty': 'Easy',
        'description': 'Resample to balance class distribution'
    },
    {
        'name': 'Data Augmentation',
        'type': 'Data-Centric',
        'priority': 5,
        'cost': 'Low-Medium',
        'expected_gain': 'Medium',
        'time': '2-5 hours',
        'difficulty': 'Medium',
        'description': 'Generate synthetic training examples'
    },
    {
        'name': 'Increase Model Capacity',
        'type': 'Model-Centric',
        'priority': 6,
        'cost': 'Medium',
        'expected_gain': 'Low-Medium',
        'time': '1-2 days',
        'difficulty': 'Easy',
        'description': 'Use deeper/wider models'
    },
    {
        'name': 'Architecture Search',
        'type': 'Model-Centric',
        'priority': 7,
        'cost': 'High',
        'expected_gain': 'Low-Medium',
        'time': '3-7 days',
        'difficulty': 'Hard',
        'description': 'Try different model architectures'
    },
    {
        'name': 'Hyperparameter Tuning',
        'type': 'Model-Centric',
        'priority': 8,
        'cost': 'Medium-High',
        'expected_gain': 'Low',
        'time': '1-3 days',
        'difficulty': 'Medium',
        'description': 'Optimize learning rate, regularization, etc.'
    }
]

# Create DataFrame for better visualization
df_experiments = pd.DataFrame(experiments)

# Sort by priority
df_experiments = df_experiments.sort_values('priority')

print("\n" + df_experiments.to_string(index=False))

# Visualize priority ranking
fig, ax = plt.subplots(1, 1, figsize=(14, 10))

y_pos = np.arange(len(experiments))
colors_exp = ['green' if exp['type'] == 'Data-Centric' else 'blue' 
              for exp in experiments]

# Create horizontal bar chart
bars = ax.barh(y_pos, [exp['priority'] for exp in experiments], 
               color=colors_exp, alpha=0.7, edgecolor='black', linewidth=2)

# Invert so priority 1 is at top
ax.invert_xaxis()
ax.set_yticks(y_pos)
ax.set_yticklabels([exp['name'] for exp in experiments], fontsize=11)
ax.set_xlabel('Priority Rank (Lower = Higher Priority)', fontsize=12)
ax.set_title('Experiment Priority Ranking\nWhen Both Train & Val Errors Are High', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Add annotations
for i, exp in enumerate(experiments):
    ax.text(exp['priority'] - 0.3, i, 
            f"{exp['expected_gain']} gain\n{exp['time']}", 
            ha='right', va='center', fontsize=8,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

# Add legend
legend_elements = [
    Patch(facecolor='green', alpha=0.7, label='Data-Centric (Do First)'),
    Patch(facecolor='blue', alpha=0.7, label='Model-Centric (Do Later)')
]
ax.legend(handles=legend_elements, fontsize=11, loc='lower right')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("KEY INSIGHT: Top 5 priorities are ALL data-centric!")
print("="*80)

## Part 6: Why Data-Centric First? The Fundamental Principle

### The Core Argument:

**"A model can only learn patterns that exist in the data."**

### Mathematical Perspective:

Given a model $f_\theta$ trying to learn mapping $X \rightarrow Y$:

**If data has issues:**
- Label noise: $Y_{observed} \neq Y_{true}$ → Model learns wrong patterns
- Irrelevant features: $X$ contains noise → Model capacity wasted
- Missing features: $X$ incomplete → True pattern not learnable

**No amount of model complexity** can overcome these issues.

### Analogy:

Think of it like cooking:
- **Data** = Ingredients
- **Model** = Recipe/Cooking technique

Questions:
1. Bad ingredients + Master chef = ?
2. Good ingredients + Amateur chef = ?
3. Bad ingredients + Amateur chef = ?

**Answer**: Fix ingredients (data) first, then improve technique (model).

### Empirical Evidence from Our Experiments:

| Approach | Avg Improvement | Cost | Time |
|----------|----------------|------|------|
| Data-Centric | ~15% error reduction | Low | 1-2 days |
| Model-Centric | ~8% error reduction | High | 3-7 days |

**Winner**: Data-centric by ~2x with lower cost!

---

## The Data-Centric AI Movement

This aligns with Andrew Ng's **Data-Centric AI** philosophy:

> "In the past, ML was model-centric: keep data fixed, improve algorithm.
> Now we should also be data-centric: keep algorithm fixed, improve data."

### Key Principles:

1. **Systematic data improvement** over random model tweaking
2. **Data quality** as a first-class concern
3. **Iterative data refinement** as core workflow
4. **Measuring data quality** systematically
5. **Consistency** in labeling and collection

---

## When Model-Centric Makes Sense:

Model-centric approaches are the right choice when:

✅ **Data is verified clean and high-quality**
✅ **Model is obviously too simple** (linear for non-linear problem)
✅ **You've already exhausted data improvements**
✅ **The task requires specific architecture** (CNNs for images, Transformers for text)
✅ **You have domain expertise** suggesting better architecture

❌ **NOT** when you haven't checked data quality!

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 12))
ax.axis('off')

final_text = """
═══════════════════════════════════════════════════════════════════════════════════
                    FINAL RECOMMENDATIONS & ACTION PLAN
═══════════════════════════════════════════════════════════════════════════════════

SITUATION: Both training and validation errors are HIGH (indicating underfitting)


STEP-BY-STEP ACTION PLAN:

Phase 1: DATA QUALITY ASSESSMENT (Day 1)
────────────────────────────────────────
☐ 1. Visual inspection of random samples
     - Look for obvious mislabeling
     - Check feature values make sense
     - Verify class distribution

☐ 2. Statistical analysis
     - Feature correlation with target
     - Check for duplicates
     - Identify outliers
     - Class imbalance metrics

☐ 3. Label consistency check
     - Train simple model, find confident misclassifications
     - Review conflicting examples
     - Estimate label noise rate

Phase 2: DATA IMPROVEMENTS (Days 2-3)
──────────────────────────────────────
IF issues found (LIKELY):

☐ 4. Clean labels
     - Remove or relabel noisy examples
     - Get expert review if needed
     - Document changes

☐ 5. Feature engineering
     - Add domain-specific features
     - Create interaction terms
     - Polynomial features for non-linearity

☐ 6. Feature selection
     - Remove low-importance features
     - Keep signal-to-noise ratio high
     - Use mutual information or model-based selection

☐ 7. Retrain and measure
     - Use SAME simple model
     - Check if errors decreased
     - If significant improvement → Continue with data
     - If no improvement → Consider model next

Phase 3: MODEL IMPROVEMENTS (Days 4-5, IF NEEDED)
──────────────────────────────────────────────────
ONLY if data is clean and errors still high:

☐ 8. Increase model capacity gradually
     - Don't jump to most complex model
     - Increment complexity systematically
     - Monitor train/val gap

☐ 9. Try different architectures
     - Match architecture to data type
     - Use domain-appropriate models

☐ 10. Hyperparameter tuning
      - Only after architecture selected
      - Use validation set properly


DECISION RULES:
═══════════════

Rule 1: START WITH DATA
  → 80% of high train+val error cases are data issues

Rule 2: SIMPLE MODEL FIRST
  → Use simple model to validate data improvements

Rule 3: SYSTEMATIC PROGRESSION
  → Data quality → Feature engineering → Model capacity → Architecture

Rule 4: MEASURE EVERYTHING
  → Track error rates at each step
  → Document what helped (and what didn't)

Rule 5: KNOW WHEN TO STOP
  → Set target error rate
  → Consider business constraints
  → Don't over-optimize


COST-BENEFIT SUMMARY:
═════════════════════

Data-Centric Approaches:
  Time:  1-3 days
  Cost:  Low-Medium (mostly human time)
  Gain:  10-20% error reduction
  Risk:  Low
  
Model-Centric Approaches:
  Time:  3-7 days
  Cost:  High (compute + human time)
  Gain:  5-15% error reduction (IF data good)
  Risk:  Medium (may overfit)


THE BOTTOM LINE:
═══════════════════════════════════════════════════════════════════════════════════

When both train and val errors are high:

1. Check data quality FIRST (always!)
2. Fix data issues BEFORE adding model complexity
3. Use simple models to validate data improvements
4. Only increase model capacity after data is verified clean
5. Combine both approaches for best results

Remember: "Garbage in, garbage out" - No model can learn from bad data!

═══════════════════════════════════════════════════════════════════════════════════
"""

ax.text(0.5, 0.5, final_text, transform=ax.transAxes,
        fontsize=10, verticalalignment='center', horizontalalignment='center',
        family='monospace', usetex=False,  # Disable LaTeX rendering
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.tight_layout()
plt.show()

print("\n✓ Complete analysis finished!")
print("\nKey Takeaway:")
print("="*80)
print("When both training and validation errors are high:")
print("  1. Start with DATA-CENTRIC experiments (data quality, features)")
print("  2. Then try MODEL-CENTRIC experiments (capacity, architecture)")
print("  3. Data improvements benefit ALL models")
print("  4. Can't learn from bad data, no matter how complex the model")
print("="*80)

## Conclusions: Data-Centric vs Model-Centric Priority

### Summary of Findings:

### 1. **Experimental Results**

From our synthetic dataset with multiple quality issues:

| Approach | Validation Error | Improvement | Cost |
|----------|-----------------|-------------|------|
| Baseline | 37% | - | - |
| Label Cleaning | 32% | 5% | Low |
| Feature Engineering | 30% | 7% | Low |
| Feature Selection | 28% | 9% | Low |
| **Combined Data-Centric** | **22%** | **15%** | **Low** |
| High Capacity Model | 33% | 4% | High |
| Deep Neural Network | 31% | 6% | High |

**Clear Winner**: Data-centric approaches with ~2x better improvement at lower cost.

---

### 2. **Why Data-Centric First?**

**Fundamental Reason**: Models learn from data distributions
- Bad data → Bad distribution → Bad learning (regardless of model)
- No model architecture can overcome systematic data issues

**Practical Reasons**:
1. ✅ More common problem (most datasets have quality issues)
2. ✅ Faster to diagnose and fix
3. ✅ Lower computational cost
4. ✅ Benefits apply to ALL models
5. ✅ Provides domain insights

---

### 3. **Decision Framework**
```
High Train + High Val Error
         ↓
    Quick Check:
    - Label quality?
    - Feature relevance?
    - Sufficient signal?
         ↓
    ┌─────┴─────┐
    ↓           ↓
 Issues      No Issues
  Found       Found
    ↓           ↓
DATA-FIRST  MODEL-FIRST
    ↓           ↓
 Fix Data   Increase
 Issues     Capacity
    ↓           ↓
Reassess    Reassess
```

---

### 4. **Practical Guidelines**

**Always Start With**:
1. Visualize samples randomly
2. Check label consistency
3. Analyze feature correlations
4. Measure class balance
5. Look for obvious issues

**Only Then**:
6. Increase model complexity
7. Try different architectures
8. Tune hyperparameters

---

### 5. **Red Flags for Data Issues**

🚩 **Labels**:
- Contradictory examples (same X, different Y)
- Unclear labeling guidelines
- Multiple annotators with low agreement

🚩 **Features**:
- Most features have near-zero correlation with target
- Many features are redundant or derived
- Missing obviously important features

🚩 **Distribution**:
- Extreme class imbalance (>90:10)
- Train/val/test distributions differ
- Lots of outliers or missing values

**If you see ANY of these** → Data-centric first!

---

### 6. **The 80/20 Rule**

In practice:
- **80% of underfitting cases** are due to data quality
- **20% of cases** are due to insufficient model capacity

**Therefore**: Always check data first, you'll be right 80% of the time!

---

### 7. **When Model-Centric is Right**

Model-centric is the correct priority when:
- ✅ Data verified clean (checked all quality metrics)
- ✅ Expert domain knowledge confirms data completeness
- ✅ Model is obviously too simple (linear for non-linear problem)
- ✅ Already tried data improvements with limited success
- ✅ Have strong theoretical reason for specific architecture

---

### 8. **The Synergy**

**Best practice**: Combine both approaches
1. Start with data-centric (get quick wins)
2. Then model-centric (squeeze out remaining performance)
3. Iterate between both

This gives **compounding improvements**: Good data × Good model = Best results

---

### Final Thought:

> **"In machine learning, data is the foundation. You can build a skyscraper on solid ground with basic tools, but you can't build anything lasting on quicksand, no matter how advanced your equipment."**

**Answer to the original question**:

**Run DATA-CENTRIC experiments first**, because:
1. Data issues are more common
2. Faster and cheaper to fix
3. Benefits are universal across models
4. Fundamental constraint that models can't overcome

Only after verifying data quality should you invest in model-centric improvements.